In [106]:
import pandas as pd
import numpy as np
import requests
import os
import glob
import shutil
import string

In [107]:
def calc_target(kpis):
    return sum(5*(x_ref - x_ours) if x_ref > x_ours else x_ref - x_ours for x_ref, x_ours in kpis)

In [108]:
# Helper functions

def getAmount(solution):
    solution2 = solution.loc[~(solution.drop(columns="amount") == 0).all(axis=1)]
    return solution2["amount"].mean() * 1


def getVariance(solution):
    solution_amountless = solution.drop(columns="amount")
    temp = {col: 0 for col in solution_amountless.columns}
    for index, nominator in solution.iterrows():
        nominator_amountless = nominator.drop("amount")
        selected = nominator_amountless[nominator_amountless==1].index.tolist()

        if len(selected) == 0: continue

        amount_to_distribute = nominator["amount"] / len(selected)

        for validator in selected:
            temp[validator] += amount_to_distribute
            
    return np.array(list(temp.values())).std() * -1


def getAssignment(solution):
    nominatorsAssignments = (solution.iloc[:, :-1] == 1).sum(axis=1)
    return ((nominatorsAssignments - 1) ** 2).sum() * -1


def concatenateSolutions(sol1, sol2):
    result = []
    for s1, s2 in zip(sol1, sol2):
        normalizer = max(s1, s2)
        result.append((s1 / normalizer, s2 / normalizer))

    return result


def calculateKpis(sol):
    solution_amount = getAmount(sol)
    solution_variance = getVariance(sol)
    solution_assignment = getAssignment(sol)

    solution_kpis = [solution_amount, solution_variance, solution_assignment]

    return solution_kpis


def compareSolutions(sol1, sol2):
    return calc_target(concatenateSolutions(calculateKpis(sol1), calculateKpis(sol2)))


def getDataBatches(nominators_filepath_pattern='data/polkadot_nominators_session_*.csv', batch_size=1, default_min=0):
    file_paths = glob.glob(nominators_filepath_pattern)
    num_batches = len(file_paths) // batch_size + (1 if len(file_paths) % batch_size != 0 else 0)
    data_batches = []
    for i in range(num_batches):
        nominators_batches = [pd.read_csv(file) for file in file_paths[i * batch_size: (i + 1) * batch_size]]
        all_data = pd.concat(nominators_batches, ignore_index=True)
        min_batch_amount = default_min if default_min == 0 else all_data["bonded_amount"].min()
        max_batch_amount = all_data["bonded_amount"].max()

        nominators_batches = list(map(lambda x: removeEmptyTargetsRow(x, min_batch_amount, max_batch_amount), nominators_batches))
        data_batches.append(nominators_batches)
    return data_batches

def removeEmptyTargetsRow(nominators_df, min_amount, max_amount):
    nominators_no_na = nominators_df[nominators_df["targets"].notna()]
    return normalizeAmountColumn(nominators_no_na, min_amount, max_amount)

def normalizeAmountColumn(nominators_df, min_amount, max_amount):
    nominators_df['bonded_amount'] = (nominators_df['bonded_amount'] - min_amount) / (max_amount - min_amount)
    return nominators_df


In [109]:
google_cloud_url = "https://storage.googleapis.com/watcher-csv-exporter/"
session_filename_template = string.Template("polkadot_nominators_session_$id.csv")
era_filename_template = string.Template("polkadot_validators_era_$id.csv")

Loading of data starting from era number 165 and session 1031

In [110]:
def download_file(url, destination):
    response = requests.get(url, stream=True)
    response.raise_for_status()
    with open(destination, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

def download_batch(starting_era=1000, starting_session=6041, destinationFolder="data/", number_of_eras_to_download=1):
    if os.path.exists(destinationFolder):
         shutil.rmtree(destinationFolder)
         
    os.makedirs(destinationFolder, exist_ok=True)
    era_id = starting_era
    session_id = starting_session
    total_eras_downloaded = 0

    while total_eras_downloaded < number_of_eras_to_download:
        era_filename = era_filename_template.substitute({'id': era_id})

        try:
            download_file(
                google_cloud_url + era_filename,
                destinationFolder + era_filename
            )
        except Exception as e:
            era_id += 1
            session_id += 6
            continue
        
        session_filename = session_filename_template.substitute({'id': session_id})
        download_file(
            google_cloud_url + session_filename,
            destinationFolder + session_filename
        )

        total_eras_downloaded += 1
        era_id += 1
        session_id += 6

download_batch()

In [111]:
nominators = getDataBatches()[0][0]
validators = pd.read_csv("data/polkadot_validators_era_1000.csv")

/tmp/ipykernel_7670/3478919451.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nominators_df['bonded_amount'] = (nominators_df['bonded_amount'] - min_amount) / (max_amount - min_amount)


In [112]:
nominators

,era,session,last_session,timestamp,block_number,stash_address,controller_address,bonded_amount,num_targets,targets
0,1000,6041,1,1677510906000,14432836,1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,1.411597e-05,1,13uW7auWPX9WAtqwkBx7yagb78PLcv8FAcPZEVCovbXoNJK4
1,1000,6041,1,1677510906000,14432836,12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,1.129135e-04,10,14Y4s6V1PWrwBLvxW47gcYgZCGTYekmmzvFsK1kiqNH2d8...
2,1000,6041,1,1677510906000,14432836,15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,5.083039e-05,16,111B8CxcmnWbuDLyGvgUmRezDCK1brRZmvUuQ6SrFdMyc3...
3,1000,6041,1,1677510906000,14432836,1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,2.402486e-05,16,11uMPbeaEDJhUxzU4ZfWW9VQEsryP9XqFcNRfPdYda6aFW...
4,1000,6041,1,1677510906000,14432836,14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,2.660759e-05,16,13uW7auWPX9WAtqwkBx7yagb78PLcv8FAcPZEVCovbXoNJ...
...,...,...,...,...,...,...,...,...,...,...
45929,1000,6041,1,1677510906000,14432836,16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,2.238295e-06,14,1BdoL1BP36SZGEKR4iX8ksou2GTnrDd5of99SWK82c3A4a...
45930,1000,6041,1,1677510906000,14432836,16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,2.783675e-07,5,1Wo6qcrh7wxc1kQY5nfixFuCAFuzkgiwau64SmrPXBE7vV...
45931,1000,6041,1,1677510906000,14432836,1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,3.890566e-06,6,1jqkeJhuoRudNTVL5dV1qZf8RQtyzcf6ZT4yyvUQbKFktr...
45932,1000,6041,1,1677510906000,14432836,14ajZ7XrZBfoENeY2YKvVc8wya84ck87GVUmjHbW91AsUwXr,14ajZ7XrZBfoENeY2YKvVc8wya84ck87GVUmjHbW91AsUwXr,5.370298e-07,1,14Y4s6V1PWrwBLvxW47gcYgZCGTYekmmzvFsK1kiqNH2d84t


In [113]:
number_of_validators = len(validators)
validators

,era,session,last_session,timestamp,block_number,active,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,stakers,num_voters,voters,era_points,total_issuance,validator_rewards_previous_era
0,1000,6041,0,undefined,14433047,1,Zug Capital / 19,1zugcag7cJVBtVRnFxv5Qftn7xKAnR6YJ9x4x3XLgGgmNnS,13eeBGJPSPQjXGfBmd2rsek8gKU1ivz7eZReiVRwwES1QzuX,10.0,0,20068075408692255,194,13pGTiJfvnhBHU48v47LtrnmTztkmgyM9XMZou5TDkUDQD...,1256,16nKU2zRg4V7kdrUatNL31jtaDQpgKSkGVRKLCKQayLBV1...,87960,12876212770375009443,3173859046589575
1,1000,6041,0,undefined,14433047,1,Blockdaemon / 3,148Ta5cWD3wekK3C6EbdDhYrdxC5e71VTKQCjmHUjE1DCG31,14zaQkCUNrZzfoCtbx8PVP95Qn4YqZNsZRah2UEVCbrwSxg8,3.0,0,20068566349730829,87,12Xt2r83z7By9LkkEFQU83BCsaRcbMUDTBsrkn34S2ANFL...,1197,14CsNB81R2PRC96HmFudUSWYHrsEMsReK2qk2Tj7AW3VDC...,86740,12876212770375009443,3173859046589575
2,1000,6041,0,undefined,14433047,1,Blockdaemon / 8,13gLE9RuaZ3vmZwVaZfsvPMNnTHPVhXKoMWjsNaP6F6ccA7s,112YyZzTGg2fZWzD9MHqnAa4hrVhv3ZLYYCS2TgLxSaDa6A,3.0,0,20067868952630962,87,12wRiLAiTnTvGpWXffSEcd2X2FNo3pkoDbFhGP5FDGJ93i...,951,14fgWbkSbksjSEwBJxkYnZpEKEwfUA27wWSJwLqQg91GbW...,105180,12876212770375009443,3173859046589575
3,1000,6041,0,undefined,14433047,1,il4r141,12rgiL4r56kPE4PuYmz8snR21isfbrcp5Vbf8VdJe2AWDuus,14ugiL4rGgJrUWjYRR6pp3VLXETRZB7rXHa5qzMEcbCSwCRF,5.0,53051575439816,20067094068993158,45,1sJc6jqNYmLUcBLuUCK2JA48Qkk96xNS3znjZW6svgUcZr...,560,14VN91rYoptq4LpaPrHnYeGbssdm1PSkxczYZpG3UJ1ECt...,68840,12876212770375009443,3173859046589575
4,1000,6041,0,undefined,14433047,1,NaN,13iTiojfEzSXLprKzvE7Sdmg8gtUD2S2Am2Xv61xrtmDHcvJ,13iTiojfEzSXLprKzvE7Sdmg8gtUD2S2Am2Xv61xrtmDHcvJ,100.0,0,17857633247033609,94,16fsKd9LE71ahXmQBeJrMtxJfhEYUiKmyAixJ4zPZTFmra...,186,15JGhKcn53E4e22PsEMzMVf9QMH53PXYmSdRJA6YGq6XKV...,87440,12876212770375009443,3173859046589575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,1000,6041,0,undefined,14433047,1,NaN,1xsSvZudWSgr65zhRzeHrHHKVzRizHMdbmDKGCkSD5UXJYE,1xsSvZudWSgr65zhRzeHrHHKVzRizHMdbmDKGCkSD5UXJYE,100.0,0,18716827492364702,1,14AoK8VSrHFxZijXhZGSUipHzZbUgo2AkmEaviD9yxTb1S...,7,14Xs22PogFVE4nfPmsRFhmnqX3RqdrUANZRaVJU7Hik8DA...,87460,12876212770375009443,3173859046589575
293,1000,6041,0,undefined,14433047,1,ZKValidator / ZKValidator 3,138X7AdRKX7myWMwoFgWfHj1WUZKKZrKTr2CZ8pGUUXTrkDu,138X7AdRKX7myWMwoFgWfHj1WUZKKZrKTr2CZ8pGUUXTrkDu,8.0,0,20068168778375645,29,143VKnUS6QyPRngaciPoHLMDWRARZNRZg1X7ovk8nJWnyo...,281,16ZaqSpoJpsFU8gid2KrP8CEPmnKRXTNJ7G2vDaSysnxft...,69800,12876212770375009443,3173859046589575
294,1000,6041,0,undefined,14433047,1,NaN,162DmeD1MHpWQhv8NPz7zbUEfCyTfG4Tuqz7P7852KiY2wXz,162DmeD1MHpWQhv8NPz7zbUEfCyTfG4Tuqz7P7852KiY2wXz,100.0,0,26247053240920129,1,16fttU3nadc7KgFwxUqLyyryUiqW5VMbVMpTQ18GzNtbK9...,7,1REEV3CUqiMMfRdeTy9apQwXVabcg2TcQhvsVFtusByZ6g...,86680,12876212770375009443,3173859046589575
295,1000,6041,0,undefined,14433047,1,openbitlab,13pYWKctR5s8vQuyZt3pxQXue4SRH9coyAS9S9z5HtogAnhs,14gTmBa737stymz7qUVMhYG8KByUZKDdSUmWSEw2WywMYf9m,5.0,58292336548732,20067037947137545,66,15UHvPeMjYLvMLqh6bWLxAP3MbqjjsMXFWToJKCijzGPM3...,412,14PyjY7RKt8wPMzT5ujuoy8hJnoJ8fM7JKXHExBf5DXmEb...,81060,12876212770375009443,3173859046589575


In [114]:
target = nominators["targets"][0]
validators[validators["stash_address"] == target]

,era,session,last_session,timestamp,block_number,active,name,stash_address,controller_address,commission_percent,self_stake,total_stake,num_stakers,stakers,num_voters,voters,era_points,total_issuance,validator_rewards_previous_era
64,1000,6041,0,undefined,14433047,1,P2P.ORG / 13,13uW7auWPX9WAtqwkBx7yagb78PLcv8FAcPZEVCovbXoNJK4,15PqHSMuDnGQjKeHuLQacEvHZMxDQ38wq98aTHcPHiVN2B2Q,2.5,0,20068282805571018,152,1LjT3xzGQXTUPgvxqgTdkwTvokEAqbQXSK9FKAFaMSebZp...,1697,1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhC...,35760,12876212770375009443,3173859046589575


Creation of binary table similar to one in presentation, so showing which validators nominator selected and his total money

In [115]:
binary_matrix = nominators["targets"].str.get_dummies(sep=',')
binary_matrix.index = nominators["stash_address"]
binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]
binary_matrix

,111B8CxcmnWbuDLyGvgUmRezDCK1brRZmvUuQ6SrFdMyc3S,1123RekaPHgWaPL5v9qfsikRemeZdYC4tvKXYuLXwhfT3NKy,1124RsfEgJEZvEq4HbtGFcpqoxnqSy79EjNZY9tzPct3AB6o,112A6wJPeDsf34nsqoAkAtQ8n74vJU8qmyKMtzX7ZPQH2kXa,112EMfAnjR92TPqp7eKNGwiDwFGS41gJ3MWr5MPyJkyidLLc,112HGqrHtL8go7naesBTkmGS5yMyUPutNSrFkbZ7mZkza6Qc,112Ji3ASaU21FGa4ymrjvyZrDv8LbW9tF5NLUksdLdJkjY7E,114SUbKCXjmb9czpWTtS3JANSmNRwVa4mmsMrWYpRG1kDH5,1155dDdp1X4F3rh35hAMoK8r4iDVdzprtRpVrScP35YPC2b,11AnciffJctDC28odTEjDVYP2yWyp6275WLbrAUHi2vJm9f,...,1zugcauBVvcNa6tpW3j6WaGRL1Wvm9oSimK2eP3VNqNLw1V,1zugcavJYzi2KErZy9CMbLANhfrFwMESgPz9q29eUCR5gTW,1zugcavYA9yCuYwiEYeMHNJm9gXznYjNfXQjZsZukF1Mpow,1zugcawsx74AgoC4wz2dMEVFVDNo7rVuTRjZMnfNp9T49po,1zugcaxRrQDr7ktb6SpjVMkywys2ysoZWBfUfs9CDPwFVHC,1zugcaxcGLmRb6wFpftx99sYRSQihqq6KLTThimEkYsaSoq,1zugcazy9vaQJkMX79DYwFdvTYGmAbjPQrYuyUNHoFnXKtA,1zugcdarkTpYvG69VFJvxCdMVovSoWPieafJgLCnWCsYCzS,1zugchKTRDgvvfeGGSf5KpVWMx17Atf956FJYaFrmxdS4T7,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.411597e-05
12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.129135e-04
15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,1,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,5.083039e-05
1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.402486e-05
14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.660759e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.238295e-06
16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.783675e-07
1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.890566e-06


In [116]:
binary_matrix_dropped = binary_matrix.drop(columns="amount")
random_solution = binary_matrix_dropped.sample(n=number_of_validators, axis=1)
random_solution

,14mrhzKbJYyjQxdLa8ThUWbW1KPx1S8Wu9H6sGptMLKpz79r,1jeB5w8XyBADtgmVmwk2stWpTyfTVWEgLo85tF7gYVxnmSw,139MMfwzrHyveFAuCS5SUaZG85qwM66pRnvoKyST3gd9Ao1F,16mZzMEGrnpSS1TwgHe4cEJuXtQgEqKLbt8UN935dBXj25G,15w9SHbuED9bDhUvkkXugeUba7cDhQ6jUdzZtGHx9HqERipn,15aeYDFLANtjEJu4YtPhe6aoycTi8g6fBeAZjeNN4ixcSrZj,16Fwe1wFveAWmUJ8WucZmu87iyp5464m9hNqvUP8oE35nwc8,1W7neFxPiA76dthShJu3ZfJFNzyBjo3SrbafYQhT3GheLbD,14Dk4PFNRV3tvTnGTU2JVKwji1Bx4C4sgSJQMfQdaoSdTxLi,12mHfQJGXreUmuxuP2u4qLeytfdK3qkSJpMKxBQm8xmor3kd,...,1zugcaaABVRXtyepKmwNR4g5iH2NtTNVBz1McZ81p91uAm8,16AtuxfMiHiqDgBZJGHSuJqrreYaoHxUh8s7u3QtUmmosyZV,1qjNXPh99ExYXmuqHgmvfmmwjjka1M21jaVDPddZtKZhzKH,1LohyWj5cEmH1teF3bAMvR4DLTJNL1PHWWz7nzHRgo59qPd,15cXT91Xe2kC1dGKyR5Ad88tnLGAFQk9HtR7e5qdbkEgbWiz,15TFYgkh93qeEDAcZpmEcKSbRwC1VuJt8X2HVMgpt1MiuCPp,1pyeBBRDsCiKHh9dBwingmJkREAmY4dLuPnvu9xX8iPr6Ro,1dotxJwuyBMMfrGgD4LRZpd3GqXeLyGX8DZg4Vh48A44cbD,163BmbjpXzPyzQWhFBW2ghLT1gErMTiuFdR5NQEADybDTrAR,12L5PhJ2CT4MujSXoHTsBRZHQym4e6WYRhpAkgNWSwAnjZTf
stash_address,,,,,,,,,,,,,,,,,,,,,
1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [117]:
random_solution['amount'] = nominators.set_index('stash_address')['bonded_amount']
random_solution

,14mrhzKbJYyjQxdLa8ThUWbW1KPx1S8Wu9H6sGptMLKpz79r,1jeB5w8XyBADtgmVmwk2stWpTyfTVWEgLo85tF7gYVxnmSw,139MMfwzrHyveFAuCS5SUaZG85qwM66pRnvoKyST3gd9Ao1F,16mZzMEGrnpSS1TwgHe4cEJuXtQgEqKLbt8UN935dBXj25G,15w9SHbuED9bDhUvkkXugeUba7cDhQ6jUdzZtGHx9HqERipn,15aeYDFLANtjEJu4YtPhe6aoycTi8g6fBeAZjeNN4ixcSrZj,16Fwe1wFveAWmUJ8WucZmu87iyp5464m9hNqvUP8oE35nwc8,1W7neFxPiA76dthShJu3ZfJFNzyBjo3SrbafYQhT3GheLbD,14Dk4PFNRV3tvTnGTU2JVKwji1Bx4C4sgSJQMfQdaoSdTxLi,12mHfQJGXreUmuxuP2u4qLeytfdK3qkSJpMKxBQm8xmor3kd,...,16AtuxfMiHiqDgBZJGHSuJqrreYaoHxUh8s7u3QtUmmosyZV,1qjNXPh99ExYXmuqHgmvfmmwjjka1M21jaVDPddZtKZhzKH,1LohyWj5cEmH1teF3bAMvR4DLTJNL1PHWWz7nzHRgo59qPd,15cXT91Xe2kC1dGKyR5Ad88tnLGAFQk9HtR7e5qdbkEgbWiz,15TFYgkh93qeEDAcZpmEcKSbRwC1VuJt8X2HVMgpt1MiuCPp,1pyeBBRDsCiKHh9dBwingmJkREAmY4dLuPnvu9xX8iPr6Ro,1dotxJwuyBMMfrGgD4LRZpd3GqXeLyGX8DZg4Vh48A44cbD,163BmbjpXzPyzQWhFBW2ghLT1gErMTiuFdR5NQEADybDTrAR,12L5PhJ2CT4MujSXoHTsBRZHQym4e6WYRhpAkgNWSwAnjZTf,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.411597e-05
12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.129135e-04
15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.083039e-05
1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.402486e-05
14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.660759e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.238295e-06
16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.783675e-07
1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.890566e-06


In [118]:
random_solution_amount = getAmount(random_solution)
random_solution_variance = getVariance(random_solution)
random_solution_assignment = getAssignment(random_solution)

random_solution_kpis = [random_solution_amount, random_solution_variance, random_solution_assignment]
random_solution_kpis

[0.0003771823570505613, -0.08235329369378484, -205978]

In [119]:
random_solution2 = binary_matrix_dropped.sample(n=number_of_validators, axis=1)
random_solution2['amount'] = nominators.set_index('stash_address')['bonded_amount']
random_solution2_amount = getAmount(random_solution2)
random_solution2_variance = getVariance(random_solution2)
random_solution2_assignment = getAssignment(random_solution2)

random_solution_kpis2 = [random_solution2_amount, random_solution2_variance, random_solution2_assignment]
random_solution_kpis2

[0.000365035884959529, -0.06454302479023474, -232027]

In [120]:
score = calc_target(concatenateSolutions(random_solution_kpis, random_solution_kpis2))
score

1.414271603935118

In [121]:
score = calc_target(concatenateSolutions(random_solution_kpis2, random_solution_kpis))
score

0.3241774981450949

In [122]:
validators["stash_address"]
ref_sol = binary_matrix[validators["stash_address"]]

ref_sol['amount'] = nominators.set_index('stash_address')['bonded_amount']
ref_sol_amount = getAmount(ref_sol)
ref_sol_variance = getVariance(ref_sol)
ref_sol_assignment = getAssignment(ref_sol)


/tmp/ipykernel_7670/3737593548.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ref_sol['amount'] = nominators.set_index('stash_address')['bonded_amount']


In [123]:
ref_sol_kpis = [ref_sol_amount, ref_sol_variance, ref_sol_assignment]
score = calc_target(concatenateSolutions(ref_sol_kpis, random_solution_kpis))
score

37.80499114364884

In [124]:
ref_sol_kpis = [ref_sol_amount, ref_sol_variance, ref_sol_assignment]
score = calc_target(concatenateSolutions(ref_sol_kpis, random_solution_kpis2))
score

33.43544132338893

## Gready Solution

In [125]:
## Focus On total amount
def solve_total_amount(nominators, num_of_vals):
    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]
    selected_validators = set()

    binary_matrix.sort_values(by='amount', inplace=True, ascending=False)
    i = 0

    while len(selected_validators) < num_of_vals:
        cols = binary_matrix.columns[binary_matrix.iloc[i] == 1].tolist()
        to_add = num_of_vals - len(selected_validators)
        if len(cols) <= to_add:
            selected_validators.update(cols)
        else:
            selected_validators.update(cols[:to_add])
        i += 1

    result = binary_matrix[list(selected_validators)]
    result['amount'] = nominators.set_index('stash_address')['bonded_amount']

    return result

In [139]:
## Focus on amount variance
def solve_variance_only(nominators, num_of_vals):
    nominators_copy = nominators.copy(deep=True)
    nominators_copy['targets'] = nominators_copy['targets'].apply(lambda x: x.split(','))
    expanded_nominators = nominators_copy.explode('targets')

    validator_stakes = expanded_nominators.groupby('targets')['bonded_amount'].sum().reset_index()
    validator_stakes = validator_stakes.rename(columns={'targets': 'validator_id', 'bonded_amount': 'total_stake'})
    validator_stakes_sorted = validator_stakes.sort_values(by='total_stake', ascending=False)

    selected_validators = []

    for _, validator in validator_stakes_sorted.iterrows():
        stakes = [v['total_stake'] for v in selected_validators] + [validator['total_stake']]
        variance = np.var(stakes)

        if len(selected_validators) < num_of_vals or variance < np.var(stakes[:-1]):
            selected_validators.append(validator)

        if len(selected_validators) >= num_of_vals:
            break

    selected_validators_df = pd.DataFrame(selected_validators)

    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]

    return binary_matrix[list(selected_validators_df['validator_id']) + ['amount']]

In [140]:
## Focus on assignments
def solve_assignments_only(nominators, num_of_vals):
    nominators_copy = nominators.copy(deep=True)
    nominators_copy['targets'] = nominators_copy['targets'].apply(lambda x: x.split(','))
    expanded_nominators = nominators_copy.explode('targets')

    validator_assignments = expanded_nominators.groupby('targets')['stash_address'].nunique().reset_index()
    validator_assignments = validator_assignments.rename(columns={'targets': 'validator_id', 'stash_address': 'num_assignments'})
    validator_assignments_sorted = validator_assignments.sort_values(by='num_assignments', ascending=False)

    selected_validators = []

    for _, validator in validator_assignments_sorted.iterrows():
        if len(selected_validators) < num_of_vals:
            selected_validators.append(validator)

        if len(selected_validators) >= num_of_vals:
            break

    selected_validators_df = pd.DataFrame(selected_validators)

    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]

    return binary_matrix[list(selected_validators_df['validator_id']) + ['amount']]

In [ ]:
## Focus on our kpis
def solve(nominators, num_of_vals):
    nominators_copy = nominators.copy(deep=True)
    print(nominators_copy[nominators_copy['targets'].apply(lambda x: isinstance(x, float))])

    nominators_copy['targets'] = nominators_copy['targets'].apply(lambda x: x.split(','))
    
    expanded_nominators = nominators_copy.explode('targets')
    expanded_nominators['bonded_amount'] = expanded_nominators['bonded_amount'].astype(float)
    
    validator_stakes = expanded_nominators.groupby('targets')['bonded_amount'].sum().reset_index()
    validator_stakes = validator_stakes.rename(columns={'targets': 'validator_id', 'bonded_amount': 'total_stake'})
    validator_stakes_sorted = validator_stakes.sort_values(by='total_stake', ascending=False)

    selected_validators = []
    selected_nominators = set()

    for _, validator in validator_stakes_sorted.iterrows():
        validator_id = validator['validator_id']
        validator_nominators = set(expanded_nominators[expanded_nominators['targets'] == validator_id]['stash_address'])
        overlap = len(validator_nominators & selected_nominators)
        
        stakes = [v['total_stake'] for v in selected_validators] + [validator['total_stake']]
        variance = np.var(stakes)
        variance_tolerance = 0.009

        if len(selected_validators) < num_of_vals or (overlap < 2 and variance - variance_tolerance < np.var(stakes[:-1])):
            selected_validators.append(validator)
            selected_nominators.update(validator_nominators)
        
        if len(selected_validators) >= num_of_vals:
            break

    selected_validators_df = pd.DataFrame(selected_validators)
    
    binary_matrix = nominators["targets"].str.get_dummies(sep=',')
    binary_matrix.index = nominators["stash_address"]
    binary_matrix["amount"] = nominators.set_index("stash_address")["bonded_amount"]

    return binary_matrix[list(selected_validators_df['validator_id']) + ['amount']]

In [ ]:
gready_amount_solution = solve_total_amount(nominators, number_of_validators)
gready_amount_solution

/tmp/ipykernel_7670/596922311.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['amount'] = nominators.set_index('stash_address')['bonded_amount']


,152rhiuVCzb2ehkTnga29tWwe8dRFKS5ERBvzSKS6ndtc7vt,15w9SHbuED9bDhUvkkXugeUba7cDhQ6jUdzZtGHx9HqERipn,16fish7rrBhVnkmfQuiKUaPZGm8AiK2oGGJoTwrL4keeHbbe,14KDJPdZ7aNatRihFevpkz8JwGnsMSbtPQ3Nm5oiNMCnJK6s,15wT5c8vYVpRTZgvYXM7UaEBiUcyUyiZw47EvQqcbUmxrD3u,13VS9jM2pkKNwd8LFpQNNRPBKUonU9jzGZ5fTEZiuB4nYF6Y,13unmJm4yBZn12rFXQjV1pirJQbYc6EzzxrYBcSxYcN31NsF,12JZr1HgK8w6zsbBj6oAEVRkvisn8j3MrkXugqtvc4E8uwLo,13BjZTpwEtPMcHrEbqxpgS8mz44cwv31UgyXoVM2uggSHKAT,13ougYD2SRkn88L14XiYCJc3mL7AzWoAMVdn1FwLumV49LjU,...,16kzYodFWy43sVGFKGnpDK8T3P8F9NhyMKqWNFfeFVN4XT9K,12eZSMk8GJYE2Lq943dSEnMUoZs1ek7jU8QN3rEA1yPZPjcR,15kkg1mK1tCGgqqo3c1CghtKCQsBEAPPjYNNmmRT3r29FeRX,139MMfwzrHyveFAuCS5SUaZG85qwM66pRnvoKyST3gd9Ao1F,12713bbq45c66CN9AD7yusSXWE1kY91DcMpjVcB2rXqZKy2w,1LMtHkfrADk7awSEFC45nyDKWxPu9cK796vtrf7Fu3NZQmB,1RJP5i7zuyBLtgGTMCD9oF8zQMTQvfc4zpKNsVxfvTKdHmr,15a9RMz1YXunLeCyphaETaPtj7caH7Pc8DVymuQKEiW6DLf4,13HtFCrxyz55KgkPWcnhHPwE8f8GmZrfXR3uC6jNrihGzmqz,144QovJpwiT9fbvGPC8K8kgMnFz53CCVNjUyt52KbNT2Zzd
stash_address,,,,,,,,,,,,,,,,,,,,,
15j4dg5GzsL1bw2U2AWgeyAk6QTxq43V7ZPbXdAmbVLjvDCK,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
16ZL8yLyXv3V3L3z9ofR1ovFLziyXaN1DPq4yffMAZ9czzBD,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
14Ns6kKbCoka3MS4Hn6b7oRw9fFejG8RH5rq5j63cWUfpPDJ,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16DGiP6jDwAfkAeqGfkUCtheKgUzTy7UeaiFFBAv8BwX3RhN,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16GMHo9HZv8CcJy4WLoMaU9qusgzx2wxKDLbXStEBvt5274B,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12Seczc2g14VKcS53sFWUnXnMwf9DBhpsJaba8PxcYs6jHUM,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12T1eWq9AD8sE5RkNbkWQtcE5tsXtvzS6y39E6ftcyAd5rAW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1226v2vBNW2nMhYFngtAqAemXHuALyq7yRYFYsEax8M2LcxK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
calculateKpis(gready_amount_solution)

[0.000315236418261952, -0.021348165284097848, -961565]

In [ ]:
compareSolutions(ref_sol, gready_amount_solution)

6.2554618366302766

In [ ]:
gready_variance_solution = solve_variance_only(nominators, number_of_validators)
gready_variance_solution

,13YDN239LTFZVFDuuyz8WTKHxMSCvEqiPQe1kyjABcRgxhNz,12MqEB7VLqKRgxCoGZXriPuo4GNYtaFiQ1tjMZhooPXAycM2,1eLUhRLQiikdt4zUrYdY4LwdwYDsStmjrqzGzC46XmuTatG,15MBzbxZeXFZxiTsMyvzKK7Hn4cqwHzC1QxT8uHYiYBWun3M,15M1146TPz478GuwVMxnFApEXhwReTfuPxenD1qDbLuTZAd6,12JZr1HgK8w6zsbBj6oAEVRkvisn8j3MrkXugqtvc4E8uwLo,1nQBeeXXhZ3gPH95X1KcLfN5JwPCpFUXR5Sc4A5gr8nCALw,12eZSMk8GJYE2Lq943dSEnMUoZs1ek7jU8QN3rEA1yPZPjcR,14CRo92REj3aXfUeonVSti1VEHgxhWbtKY9hwxvD5T3BBXkK,16Fwe1wFveAWmUJ8WucZmu87iyp5464m9hNqvUP8oE35nwc8,...,14vucjYRyE9kwN2CXWwmzJ1W3x6yCG5Mwj4Cs5BS6zrJLQ8r,1zugcabYjgfQdMLC3cAzQ8tJZMo45tMnGpivpAzpxB4CZyK,14BeKcfcvJSJjvu9GZ2CA8EQ3XkK9J1HdwDrfz5Sg5ERDnrP,13s9RrQSFbnp2TneY7nkdLmGc3ijBw12YQct8pc8km36Z9hg,128iAScPNNZcoSXQuFp1VkgW376KqvZs61g9Y36MuUX78ZZ6,153YD8ZHD9dRh82U419bSCB5SzWhbdAFzjj4NtA5pMazR2yC,14kpNbU4XjEHfYdqp95Gq3NkBWbgFd6J8Yjd2SneWNzvf1Yp,145Vw57NN3Y4tqFNidLTmkhaMLD4HPoRtU91vioXrKcTcirS,13bptFJSHHLTBDHupxCRR5ErnLSeHFyeUEU5ufNnBL3JeMpG,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.411597e-05
12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.129135e-04
15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.083039e-05
1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.402486e-05
14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2.660759e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.238295e-06
16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.783675e-07
1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.890566e-06


In [ ]:
calculateKpis(gready_variance_solution)

[0.0003076882163062419, -0.015802109308796698, -1939641]

In [ ]:
compareSolutions(ref_sol, gready_variance_solution)

3.9694554773409507

In [ ]:
gready_assignemts_solution = solve_assignments_only(nominators, number_of_validators)
gready_assignemts_solution

,1REAJ1k691g5Eqqg9gL7vvZCBG7FCCZ8zgQkZWd4va5ESih,1ufRSF5gx9Q8hrYoj7KwpzQzDNqLJdbKrFwC6okxa5gtBRd,11uMPbeaEDJhUxzU4ZfWW9VQEsryP9XqFcNRfPdYda6aFWJ,14QBQABMSFBsT3pDTaEQdshq7ZLmhzKiae2weZH45pw5ErYu,1jqkeJhuoRudNTVL5dV1qZf8RQtyzcf6ZT4yyvUQbKFktr8,1dGsgLgFez7gt5WjX2FYzNCJtaCjGG6W9dA42d9cHngDYGg,1Wo6qcrh7wxc1kQY5nfixFuCAFuzkgiwau64SmrPXBE7vVf,1RG5T6zGY4XovW75mTgpH6Bx7Y6uwwMmPToMCJSdMwdm4EW,14Y4s6V1PWrwBLvxW47gcYgZCGTYekmmzvFsK1kiqNH2d84t,13mK8AssyPekT5cFuYQ7ijKNXcjHPq8Gnx6TxF5eFCAwoLQ,...,15yiAFuYzks3FGG8cTc2ukw86JCYKrZqKkmStVhTL4hv77XV,13VS9jM2pkKNwd8LFpQNNRPBKUonU9jzGZ5fTEZiuB4nYF6Y,15ictvkBL2D3aWxyoqh8roJkRC1tdFw3SCLqjyssjuf6yiC9,16JcdktXy1ykqPPtQsu4jrtSg7nBfBxKShe64oLxJkeaEb5h,16WXKsa3jddMq8MTM671LQjUDYHprnrwEpZHyqCrwBTU3Vzk,13pndCL5D93GXZA6vuuHjvbnGSLYoaEMvTdrDQjSG7M3wYC7,1x9GAG2hXoMt3i7jL7vTZLpdhNGyjffgQUwe8GU9HhhH62e,12dGS1zjyiUqj7GuxDDwv9i72RMye1mT7tSWNaSx7QVeJ32H,12C9U6zSSoZ6pgwR2ksFyBLgQH6v7dkqqPCRyHceoP8MJRo2,amount
stash_address,,,,,,,,,,,,,,,,,,,,,
1vfh4WtfRERYNY9mkEyCpEPYXPqLqESFM8XuHFRELW6qhCB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.411597e-05
12zr2Sd7UvujakqUTJRxW3bT7bMQbn8iaRWxhN7RyAfhRPZY,1,0,0,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.129135e-04
15Xn1yd7dwWe2ohcG8oyMY2u6LQsfDrVgj7cQdjL4KPAvBsy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5.083039e-05
1MZ2PGw1z3tqWFTHy3W7gaZd9qPfconW5AY21QSC2SYQ4Dj,0,1,1,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,2.402486e-05
14AaafgpW28dMWv4ESMd2TLMdejQR7TR5tUQkVqmwC4hNgoa,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2.660759e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16c5asNZWgjwKc1HUvVYKXsTqQ1mBRh1azeingM77G48qtip,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.238295e-06
16ipEr7ZAVpNZ6TPJK7ZzhBJYH2YJQ4kjPUvFLbgxoTvTyJW,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2.783675e-07
1cdhsnffi25hyNyDkTTSL3HRGWKbELcEfJHGHimemYwYsUi,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.890566e-06


In [ ]:
calculateKpis(gready_assignemts_solution)

[0.00018045443598160076, -0.048467520095124984, -3763213]

In [ ]:
compareSolutions(ref_sol, gready_assignemts_solution)

0.44391244617738246

In [157]:
gready_solution = solve(nominators, number_of_validators)
gready_solution_kpis = calculateKpis(gready_solution)
print(gready_solution_kpis)
compareSolutions(ref_sol, gready_solution)

Empty DataFrame
Columns: [era, session, last_session, timestamp, block_number, stash_address, controller_address, bonded_amount, num_targets, targets]
Index: []
[0.0003076882163062419, -0.015802109308796698, -1939641]


3.9694554773409507